### Widget f&uuml;r M&uuml;hle

In [1]:
from get_closest import get_closest
from muehle import Game
from ipycanvas import MultiCanvas

# from ipywidgets import Output
# debug = Output(layout = {'border': '1px solid black'})

class Widget:
    
    y_offset = 50
    pos_status = (20, 20)
    line_width = 2
   
    scale = 0.8
    pts =  [(-1, -1), (0, -1), (1, -1), (1, 0), (1, 1), (0, 1), (-1, 1), (-1, 0)]
     
        
    def __init__(self, game, width=200, height=200, color_0='red', color_1='blue'):
        self.colors = {0: color_0, 1: color_1}
        self.radius = min(width, height) / 30
        
        self.mcanvas = MultiCanvas(3, width = width, height = height, 
                layout = {'border' : '1px solid black'},
               )
        self.mcanvas.on_mouse_down(self.on_mouse_down)
        self.mcanvas.on_mouse_up(self.on_mouse_up)
        self.mcanvas.on_key_down(self.on_key_down)
        
        self.bg, self.fg, self.info = self.mcanvas
        self.bg.line_width = self.line_width
       
        self.ris = [(r,i) for r in range(3) for i in range(8)]
        self.xys = [self.ri2xy(ri) for ri in self.ris]
        self.selected_idx = None
        
        self.game = game
        self.game.callback = self.update
        self.game.new_game()
        
    def ri2xy(self, ri):
        r, i = ri
        s, w, h = self.scale, self.mcanvas.width, self.mcanvas.height - self.y_offset
        x, y = self.pts[i]
        x = (s/3*(r+1)*x + 1) * w/2
        y = (s/3*(r+1)*y + 1) * h/2 + self.y_offset
       
        return x, y    
        
    # @debug.capture()        
    def on_mouse_down(self, x, y):
        if (idx := get_closest(self.xys, (x, y))) is None:
            return
        
        src = self.ris[idx]
        ptm = self.game.ptm
        
        if self.game.muehle:
            self.game.move(ptm, 'r', src)
            print(ptm, 'remove', src)
        elif self.game.phase == 'place':
            self.game.move(ptm, 'p', src)
            print(ptm, 'place', src)
        else:
            self.selected_idx = idx
                
    # @debug.capture()   
    def on_mouse_up(self, x, y):   
        if self.selected_idx is None or self.game.phase != 'move':
            return
        
        idx = get_closest(self.xys, (x, y))
        if idx is not None:
            target = self.ris[idx]
            src = self.ris[self.selected_idx]
            ptm = self.game.ptm
            self.game.move(ptm, 'm', src, target)
            print(ptm, src, target)
           
        self.selected_idx = None 
    
    # @debug.capture()   
    def on_key_down(self, key, *flags):   
        if key in '3456789':
            self.game.new_game(int(key))
       
    def new_game(self, muehlen):
        self.mcanvas.clear()
        for muehle in muehlen:
            line = [self.ri2xy(ri) for ri in muehle]
            self.bg.stroke_lines(line)
     
    def place_stone(self, player, src):
        color = self.colors[player]
        x, y = self.ri2xy(src)
        self.fg.fill_style = color
        self.fg.fill_circle(x, y, self.radius)
    
    def remove_stone(self, src):
        x, y = self.ri2xy(src)
        x -= (self.radius + 1)
        y -= (self.radius + 1)
        self.fg.clear_rect(x, y, 2*self.radius + 2)
        
    def move_stone(self, player, src, target):
        self.remove_stone(src)
        self.place_stone(player, target)
        
    def show_status(self, ptm, phase, muehle, stones_left, winner=None):
        self.info.clear()
        self.info.fill_style = self.colors[ptm]
       
        if winner is not None:
            msg = 'You won!'
        elif muehle:
            msg = 'pick a stone to remove'
        elif phase == 'place':
            msg = 'place a stone (stones left:{})'.format(stones_left)
        elif phase == 'move':
            msg = 'move a stone'    
            
        self.info.fill_text(msg, *self.pos_status)
        
    # @debug.capture()      
    def update(self, event, data):
        #print(event, data)
        getattr(self, event)(**data)
    
    def _ipython_display_(self):
        #display(self.mcanvas, debug)
        display(self.mcanvas)
        self.mcanvas.focus()

In [2]:
game = Game()
Widget(game)

MultiCanvas(height=200, layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_r…